In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import os

In [3]:
#Load face and eye cascade first
face_cascade=cv2.CascadeClassifier("./model/haarcascades/haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier('./model/haarcascades/haarcascade_eye.xml')
eye_cascade
face_cascade


< cv2.CascadeClassifier 000002574919E590>

In [4]:
#function to return croped image of face with 2 or more eyes
def get_cropped_image_if_2_eyes(image_path):
    try:
        img = cv2.imread(image_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_gray)
            if len(eyes) >= 2:
                return roi_color
    except:
        return None

In [5]:
path_of_data = "./model/dataset/rawData/"
path_to_cr_data = "./model/dataset/cropped/"

In [6]:
stu_folder_list=os.listdir(path_of_data)
stu_folder_list

['Aditya Sharma',
 'Akansha Shishodia',
 'Anant Jain',
 'Deepak Singhal',
 'Divyanka Mishra',
 'Nishant Rohilla',
 'Prashant Saraswat',
 'Prerna Sharma',
 'Ritik Kamboj',
 'Ritika Tiwari',
 'Sakshi',
 'Shubham',
 'Vansh Saini']

In [7]:
# To extract facial data from raw data 
stu_file_names_dict = {}
for sName in stu_folder_list:
    stu_file_names_dict[sName]=[]
    NewDir=os.mkdir(path_to_cr_data+sName+" cropped")
    count=1
    for sNamePhoto in os.listdir(path_of_data+sName):
        sNamePhotoPath=path_of_data+sName+"/"+sNamePhoto
       # print(sNamePhotoPath)
        Ret_Img_Color=get_cropped_image_if_2_eyes(sNamePhotoPath)
        if Ret_Img_Color is not None :
           # print(sNamePhotoPath)
            Ret_Img_Path=(path_to_cr_data+sName+" cropped")+"/"+sName+str(count)+".png"
            cv2.imwrite(Ret_Img_Path,Ret_Img_Color)
            stu_file_names_dict[sName].append(Ret_Img_Path)
            
            count+=1
    

FileExistsError: [WinError 183] Cannot create a file when that file already exists: './model/dataset/cropped/Aditya Sharma cropped'

In [ ]:
stu_file_names_dict

In [ ]:
import pywt #for wavelet function
def w2d(img, mode='haar', level=1):
    imArray = img
      #print(imArray.shape)
    #Datatype conversions
    #convert to grayscale
    imArray = cv2.cvtColor( imArray,cv2.COLOR_RGB2GRAY )
    #convert to float
    imArray =  np.float32(imArray)   
    #print(imArray)
    
    imArray /= 255;
   # print(imArray)
    # compute coefficients 
    coeffs=pywt.wavedec2(imArray, mode, level=level)
   # print(coeffs)
    #Process Coefficients
    coeffs_H=list(coeffs)  
   # print(coeffs_H)
    coeffs_H[0] *= 0;  
    print(coeffs_H)

    # reconstruction
    imArray_H=pywt.waverec2(coeffs_H, mode);
    imArray_H *= 255;
    imArray_H =  np.uint8(imArray_H)

    return imArray_H

In [ ]:
class_dict = {}
count = 0
for stu_name in stu_file_names_dict.keys():
    class_dict[stu_name] = count
    count = count + 1
class_dict

In [ ]:
X,y=[],[]
for stu_name, training_files in stu_file_names_dict.items():
    for training_image in training_files:
        img = cv2.imread(training_image)
        scalled_raw_img = cv2.resize(img, (200, 200))
        img_har = w2d(img,'db1',5)
        scalled_img_har = cv2.resize(img_har, (200, 200))
        #plt.imshow(img_har,cmap='gray')
        combined_img = np.vstack((scalled_raw_img.reshape(200*200*3,1),scalled_img_har.reshape(200*200,1)))
        X.append(combined_img)
        y.append(class_dict[stu_name]) 

In [ ]:
len(X)

In [ ]:
len(y)

In [ ]:
X = np.array(X).reshape(len(X),160000).astype(float)
X.shape

In [ ]:
X[0].shape

In [ ]:
"""
#test
path_tst_img=path_of_data+stu_folder_list[0]+"/"+os.listdir(path_of_data+stu_folder_list[0])[0]
print(os.listdir(path_of_data+stu_folder_list[0])[0])
print(path_tst_img)
#Test=cv2.imread(path_tst_img)
#plt.imshow(Test)
#ret_img=get_cropped_image_if_2_eyes(path_tst_img)
img = cv2.imread(path_tst_img)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.imshow(gray,cmap='gray')
faces = face_cascade.detectMultiScale(gray, 1.3, 5)
(x,y,w,h)=faces[0]
print(x,y,w,h)
roi_gray = gray[y:y+h, x:x+w]
plt.imshow(roi_gray)
roi_color = img[y:y+h, x:x+w]
 #           eyes = eye_cascade.detectMultiScale(roi_gray)
  #          if len(eyes) >= 2:
   #             return roi_color
#plt.imshow(ret_img)
"""

# Let's train the model


In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
len(y_test),X_test.shape

In [ ]:
svm_model=svm.SVC(C=10)
svm_model.fit(X_train,y_train)
svm_model.predict(X_test)==y_test

In [ ]:
svm_model.score(X_test,y_test)

In [ ]:
# For Test
import cv2 
img1 = cv2.imread("./model/testImg/testRt.png")
plt.imshow(img1)
faces = face_cascade.detectMultiScale(img1, 1.3, 5)
(x,y,w,h)=faces[0]
print(x,y,w,h)
img1 = img1[y:y+h, x:x+w]
scalled_raw_img1 = cv2.resize(img1, (200, 200))
img_har1 = w2d(img1,'db1',5)
scalled_img_har1 = cv2.resize(img_har1, (200, 200))
#plt.imshow(scalled_raw_img,cmap='gray')
combined_img1 = np.vstack((scalled_raw_img1.reshape(200*200*3,1),scalled_img_har1.reshape(200*200,1)))

In [ ]:
list1=[]
list1.append(combined_img1)
list1 = np.array(list1).reshape(len(list1),160000).astype(float)
list1.shape

In [ ]:
result=svm_model.predict(list1)
for k,v in class_dict.items():
    if v==result[0]:
        print(k)

In [ ]:
import joblib
joblib.dump(svm_model,"saved_model_for_SNC.pkl")

In [ ]:
import json
with open("stu_class_dict.json","w") as f:
    f.write(json.dumps(class_dict))